# Visualizing Neuropixel Probe Locations
It can be handy to know the location and trajectory of the probes that obtain the data. Some NWB files have an **electrodes** field which store these locations. To be more precise, they contain arrays of *CCF* coordinates. [CCF](https://community.brain-map.org/t/allen-mouse-ccf-accessing-and-using-related-data-and-tools/359) is a framework which represents locations in the brain with coordinates that are relative to brain structure. This notebook uses CCF coordinate data in an extracellular electrophysiology NWB file to render the locations of the [Neuropixel](https://www.neuropixels.org/probe) probes that were used.

To be able to render locations in a brain, you don't need to know how CCF works except that it is a system of coordinates. We use the **[ccf-widget](https://github.com/NeurodataWithoutBorders/ccf-widget)** library to 3D render the brain and the probe coordinates.

### Environment Setup

In [1]:
### if running on Google Colab, run this cell once, then restart the runtime and run the rest of the notebook
import os
if "COLAB_GPU" in os.environ:
    !git clone https://github.com/AllenInstitute/openscope_databook.git
    %cd openscope_databook
    %pip install -e .

In [2]:
import numpy as np

from ccfwidget import CCFWidget
from dandi_utils import dandi_download_open

### Downloading Ecephys File
If you don't already have a file to analyze, you can use a file from The Allen Institute's **Visual Coding - Neuropixels** dataset. If you want to choose your own file to download, set `dandiset_id` and `dandi_filepath` accordingly.

In [3]:
dandiset_id = "000021"
dandi_filepath = "sub-730756767/sub-730756767_ses-757970808.nwb"
download_loc = "."

In [4]:
# This can sometimes take a while depending on the size of the file
io = dandi_download_open(dandiset_id, dandi_filepath, download_loc)
nwb = io.read()

A newer version (0.55.1) of dandi/dandi-cli is available. You are using 0.46.6


File already exists
Opening file


Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.5.1 is already loaded.
Ignoring cached namespace 'core' version 2.2.2 because version 2.5.0 is already loaded.


### Extracting NWB CCF Coordinates
Here, you can read the NWB file you're interested in viewing. Specify your file of interest's relative file path in `nwb_filepath`. From there, the file will be read and the probe unit coordinates will be extracted and turned into a numpy array.

Note that this will only work with ecephys NWB files which have a valid *electrodes* field.

In [5]:
### read the x,y,z ccf coordinates and generate points
xs = nwb.electrodes.x
ys = nwb.electrodes.y
zs = nwb.electrodes.z
n = min(len(xs), len(ys), len(zs))
points = np.array([[xs[i], ys[i], zs[i]] for i in range(n)])

In [6]:
print(points.shape)

(1920, 3)


### Rendering
Rendering is as simple as generating the widget and displaying it. This will create embedded window with the interactive 3D rendering of your scene. 

In [7]:
ccf = CCFWidget(markers=[points])

In [8]:
ccf

CCFWidget(children=(VBox(children=(Viewer(background=(0.85, 0.85, 0.85), camera=array([[ 1.3441567e+03, -2.172…